In [ ]:
from re import sub
from pandas import DataFrame

from agyw_actif import actif_served as AGYW_ACTIF

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [9]:
AGYW_ACTIF.dtypes

id_patient                                         int64
departement                                       object
commune                                           object
nbre_pres_for_inter                                int16
has_comdom_topic                                  object
number_of_condoms_sensibilize                      int16
number_condoms_reception_in_the_interval           int16
number_test_date_in_the_interval                   int16
number_vbg_treatment_date_in_the_interval          int16
number_gynecological_care_date_in_the_interval     int16
number_prep_initiation_date_in_the_interval        int16
age_in_year                                        int64
age_range                                         object
month_in_program_range                            object
muso                                              object
gardening                                         object
has_a_service_with_date_in_the_past               object
curriculum_detailed            

In [10]:
pipo = list(AGYW_ACTIF.month_in_program_range.unique())
pipo.append("Total")
sorted(list(AGYW_ACTIF.age_range.unique())[0:3])
sorted(pipo)

['10-14', '15-19', '20-24']

['0-6 months', '07-12 months', '13-24 months', '25+ months', 'Total']

# Indicator

In [50]:
class AgywPrev:
    """A class with properties and methods given the results of the indicator AGYW_PREV DATIM""" 
    __who_am_I ="DATIM"
    __datim1_title="Number of individual AGYW that have fully completed the entire DREAMS primary package of services but have not received any services beyond the primary package."
    __datim2_title="Number of individual AGYW that have fully completed the entire DREAMS primary package of services AND at least one additional secondary service."
    __datim3_title="Number of individual AGYW that have fully completed at least one DREAMS primary package of services but not the full primary package of services."
    __datim4_title="Number of AGYW that have started a DREAMS service/intervention but have not yet completed it."
    
    def __repr__(self):
        return f"<AgywPrev {self.__i_am}>"
    
    def __str__(self):
        return f"<AgywPrev {self.__i_am}>"
    
    @classmethod
    def datim_titleI(cls):
        return cls.__datim1_title
    
    @classmethod
    def datim_titleII(cls):
        return cls.__datim2_title
    
    @classmethod
    def datim_titleIII(cls):
        return cls.__datim3_title
    
    @classmethod
    def datim_titleIV(cls):
        return cls.__datim4_title
    
    
    def __init__(self,commune=None):
        self.__commune = commune
        self.__i_am = f"{AgywPrev.__who_am_I}"
        self.__data =  AGYW_ACTIF
        self. __total_mastersheet = self.__data.id_patient.count()
        if self.__commune == None:
            self.__dreams_valid = self.__data[(self.__data.age_range !="not_valid_age") & (self.__data.age_range != "25-29")]
        else:
            self.__dreams_valid = self.__data[(self.__data.age_range !="not_valid_age") & (self.__data.age_range != "25-29") &(self.__data.commune == f"{self.__commune}")]
        self.__total_dreams_valid = self.__dreams_valid.id_patient.count()
        self.__dreams_valid["primary_only"] = self.__dreams_valid.apply(lambda df: self.__primFunc(df),axis=1)
        self.__dreams_valid["primary_and_OneSecondary_services"] = self.__dreams_valid.apply(lambda df: self.__primLeastOneSecFunc(df),axis=1)
        self.__dreams_valid["completed_one_service"] = self.__dreams_valid.apply(lambda df: self.__primPartFunc(df),axis=1)
        self.__dreams_valid["has_started_one_service"] = self.__dreams_valid.apply(lambda df: self.__hasStartedFunc(df),axis=1)
        self.__agyw_prevI = self.__dreams_valid[self.__dreams_valid.primary_only == "full_primary_only"]
        self.__agyw_prevII = self.__dreams_valid[self.__dreams_valid.primary_and_OneSecondary_services == "full_primary_leastOneSecondary"]
        self.__agyw_prevIII = self.__dreams_valid[self.__dreams_valid.completed_one_service == "primary_part_services"]  
        self.__agyw_prevIV = self.__dreams_valid[self.__dreams_valid.has_started_one_service == "yes"]  
        self.__agyw_prevI_total = self.__agyw_prevI.id_patient.count()
        self.__agyw_prevII_total = self.__agyw_prevII.id_patient.count()
        self.__agyw_prevIII_total = self.__agyw_prevIII.id_patient.count()
        self.__agyw_prevIV_total = self.__agyw_prevIV.id_patient.count()
        self.__total_datim = self.__agyw_prevI_total + self.__agyw_prevII_total + self.__agyw_prevIII_total + self.__agyw_prevIV_total
        
        
    @property
    def who_am_i(self):
        return self.__i_am
    
    @property
    def data_mastersheet(self):
        return self.__data
    
    @property
    def data_dreams_valid(self):
        return self.__dreams_valid

    @property
    def total_mastersheet(self):
        return self.__total_mastersheet

    @property
    def total_dreams_valid(self):
        return self.__total_dreams_valid
    
    def __primFunc(self,df):
        if (df.ps_1014 == "primary" and df.hts == "no" and df.prep=="no" and df.condom == "no" and df.post_violence_care == "no" and df.socioeco_app == "no" ): 
            return "full_primary_only"
        elif (df.ps_1519 == "primary" and df.hts == "no" and df.prep=="no"  and df.post_violence_care == "no" and df.socioeco_app == "no" ):
            return "full_primary_only"
        elif (df.ps_2024 == "primary" and df.hts == "no" and df.prep=="no"  and df.post_violence_care == "no" and df.socioeco_app == "no" ):
            return "full_primary_only"
        else:
            return "invalid"
    
    def __primLeastOneSecFunc(self,df):
        if (df.ps_1014 == "primary") and (df.hts == "yes" or df.prep=="yes" or df.condom == "yes" or df.post_violence_care == "yes" or df.socioeco_app == "yes" ):
            return "full_primary_leastOneSecondary"
        elif (df.ps_1519 == "primary") and (df.hts == "yes" or df.prep=="yes" or df.post_violence_care == "yes" or df.socioeco_app == "yes" ):
            return "full_primary_leastOneSecondary"
        elif (df.ps_2024 == "primary") and (df.hts == "yes" or df.prep=="yes"  or df.post_violence_care == "yes" or df.socioeco_app == "yes" ):
            return "full_primary_leastOneSecondary"
        else:
            return "invalid"
        
    def __primPartFunc(self,df):
        if (df.age_range=="10-14") and (df.primary_only=="invalid") and (df.primary_and_OneSecondary_services=='invalid') and ((df.hts == "yes" or df.prep=="yes" or df.condom == "yes" or df.post_violence_care == "yes" or df.socioeco_app == "yes" )):
            return "primary_part_services"
        elif (df.age_range == "15-19") and (df.primary_only=="invalid") and (df.primary_and_OneSecondary_services=='invalid') and (df.curriculum=="yes" or df.condom=="yes" or df.hts == "yes" or df.prep=="yes" or df.post_violence_care == "yes" or df.socioeco_app == "yes" ) : 
            return "primary_part_services"
        elif (df.age_range == "20-24") and (df.primary_only=="invalid")and (df.primary_and_OneSecondary_services=='invalid') and(( df.curriculum=="yes" or df.condom=="yes" or df.hts == "yes" or df.prep=="yes"  or df.post_violence_care == "yes" or df.socioeco_app == "yes" )):
            return "primary_part_services"
        else:
            return "invalid"
    
    def __hasStartedFunc(self,df):
        if (df.age_range=="10-14") and (df.primary_only=="invalid") and (df.primary_and_OneSecondary_services=='invalid') and (df.completed_one_service=="invalid"):
            return "yes"
        elif (df.age_range == "15-19") and (df.primary_only=="invalid") and (df.primary_and_OneSecondary_services=='invalid') and (df.completed_one_service=="invalid") : 
            return "yes"
        elif (df.age_range == "20-24") and (df.primary_only=="invalid")and (df.primary_and_OneSecondary_services=='invalid') and (df.completed_one_service=="invalid"):
            return "yes"
        else:
            return "no"
            
    @property
    def total_datimI(self):
        return self.__agyw_prevI_total
    
    @property
    def total_datimII(self):
        return self.__agyw_prevII_total
    
    @property
    def total_datimIII(self):
        return self.__agyw_prevIII_total
    
    @property
    def total_datimIV(self):
        return self.__agyw_prevIV_total
    
    @property
    def total_datim_general(self):
        return self.__total_datim
    
    @property
    def data_agyw_prevI(self):
        return self.__agyw_prevI

    @property
    def data_agyw_prevII(self):
        return self.__agyw_prevII

    @property
    def data_agyw_prevIII(self):
        return self.__agyw_prevIII
    
    @property
    def data_agyw_prevIV(self):
        return self.__agyw_prevIV

    __PERIOD_DATIM = sorted(list(AGYW_ACTIF.month_in_program_range.unique()))
    __PERIOD_DATIM.append("Total")
    __AGE_DATIM = sorted(list(AGYW_ACTIF.age_range.unique())[0:3])
    
    def datim_agyw_prevI(self):
        
        try:
            pivotableI = self.__agyw_prevI.rename(columns={"age_range":"Age", "month_in_program_range":"Time"})
            agyw_prevI_pivot = pivotableI.pivot_table(index="Age",columns="Time", values="id_patient",aggfunc="count",fill_value=0,margins=True,margins_name="Total",dropna=False)[:-1]
            columns_pivotI = list(agyw_prevI_pivot.columns)
            indexes_pivotI = list(agyw_prevI_pivot.index)
            for period in AgywPrev.__PERIOD_DATIM:
                if period not in columns_pivotI:
                    agyw_prevI_pivot[period] = 0
            for age in AgywPrev.__AGE_DATIM:
                if age not in indexes_pivotI:
                    agyw_prevI_pivot.loc[age] = 0
            agyw_prevI_pivot = agyw_prevI_pivot.reindex(index=AgywPrev.__AGE_DATIM, columns =AgywPrev.__PERIOD_DATIM)
            agyw_prevI_pivot_final = agyw_prevI_pivot.reset_index().rename_axis(None, axis=1)
            agyw_prevI_results_final = DataFrame( agyw_prevI_pivot_final.to_records(index=False))
        except ValueError:
            agyw_prevI_results_final = DataFrame({"Age":["10-14","15-19",
                "20-24"],
                "0-6 months":[0,0,0],
                "07-12 months":[0,0,0],
                "13-24 months":[0,0,0],
                "25+ months":[0,0,0],
                "Total":[0,0,0]
            })
        return agyw_prevI_results_final
    
    def datim_agyw_prevII(self):
        try:
            pivotableII = self.__agyw_prevII.rename(columns={"age_range":"Age", "month_in_program_range":"Time"})
            agyw_prevII_pivot = pivotableII.pivot_table(index="Age",columns="Time", values="id_patient",aggfunc="count",fill_value=0,margins=True,margins_name="Total",dropna=False)[:-1]
            columns_pivotII = list(agyw_prevII_pivot.columns)
            indexes_pivotII = list(agyw_prevII_pivot.index)
            for period in AgywPrev.__PERIOD_DATIM:
                if period not in columns_pivotII:
                    agyw_prevII_pivot[period] = 0
            for age in AgywPrev.__AGE_DATIM:
                if age not in indexes_pivotII:
                    agyw_prevII_pivot.loc[age] = 0
            agyw_prevII_pivot = agyw_prevII_pivot.reindex(index=AgywPrev.__AGE_DATIM,columns =AgywPrev.__PERIOD_DATIM)            
            agyw_prevII_pivot_final = agyw_prevII_pivot.reset_index().rename_axis(None, axis=1)
            agyw_prevII_results_final = DataFrame( agyw_prevII_pivot_final.to_records(index=False))
        except ValueError:
            agyw_prevII_results_final = DataFrame({"Age":["10-14", "15-19",
                "20-24"],
                "0-6 months":[0,0,0],
                "07-12 months":[0,0,0],
                "13-24 months":[0,0,0],
                "25+ months":[0,0,0],
                "Total":[0,0,0]
            })
        return agyw_prevII_results_final
    
    def datim_agyw_prevIII(self):
        try:
            pivotableIII = self.__agyw_prevIII.rename(columns={"age_range":"Age", "month_in_program_range":"Time"})
            agyw_prevIII_pivot = pivotableIII.pivot_table(index="Age",columns="Time", values="id_patient",aggfunc="count",fill_value=0,margins=True,margins_name="Total",dropna=False)[:-1]
            columns_pivotIII = list(agyw_prevIII_pivot.columns)
            indexes_pivotIII = list(agyw_prevIII_pivot.index)
            for period in AgywPrev.__PERIOD_DATIM:
                if period not in columns_pivotIII:
                    agyw_prevIII_pivot[period] = 0
            for age in AgywPrev.__AGE_DATIM:
                if age not in indexes_pivotIII:
                    agyw_prevIII_pivot.loc[age] = 0
            agyw_prevIII_pivot = agyw_prevIII_pivot.reindex(index=AgywPrev.__AGE_DATIM,columns =AgywPrev.__PERIOD_DATIM)
            agyw_prevIII_pivot_final = agyw_prevIII_pivot.reset_index().rename_axis(None, axis=1)
            agyw_prevIII_results_final = DataFrame( agyw_prevIII_pivot_final.to_records(index=False))
        except ValueError:
            agyw_prevIII_results_final = DataFrame({"Age":["10-14",                     "15-19",
                "20-24"],
                "0-6 months":[0,0,0],
                "07-12 months":[0,0,0],
                "13-24 months":[0,0,0],
                "25+ months":[0,0,0],
                "Total":[0,0,0]
            })
        return agyw_prevIII_results_final
    
    def datim_agyw_prevIV(self):
        try:
            pivotableIV = self.__agyw_prevIV.rename(columns={"age_range":"Age", "month_in_program_range":"Time"})
            agyw_prevIV_pivot = pivotableIV.pivot_table(index="Age",columns="Time", values="id_patient",aggfunc="count",fill_value=0,margins=True,margins_name="Total",dropna=False)[:-1]
            columns_pivotIII = list(agyw_prevIV_pivot.columns)
            indexes_pivotIII = list(agyw_prevIV_pivot.index)
            for period in AgywPrev.__PERIOD_DATIM:
                if period not in columns_pivotIII:
                    agyw_prevIV_pivot[period] = 0
            for age in AgywPrev.__AGE_DATIM:
                if age not in indexes_pivotIII:
                    agyw_prevIV_pivot.loc[age] = 0
            agyw_prevIV_pivot = agyw_prevIV_pivot.reindex(index=AgywPrev.__AGE_DATIM,columns =AgywPrev.__PERIOD_DATIM)
            agyw_prevIV_pivot_final = agyw_prevIV_pivot.reset_index().rename_axis(None, axis=1)
            agyw_prevIV_results_final = DataFrame( agyw_prevIV_pivot_final.to_records(index=False))
        except ValueError:
            agyw_prevIV_results_final = DataFrame({"Age":["10-14",                     "15-19",
                "20-24"],
                "0-6 months":[0,0,0],
                "07-12 months":[0,0,0],
                "13-24 months":[0,0,0],
                "25+ months":[0,0,0],
                "Total":[0,0,0]
            })
        return agyw_prevIV_results_final

In [51]:
%%capture
datim = AgywPrev()

In [53]:
datim.total_mastersheet 
datim.total_dreams_valid
datim.who_am_i
datim.total_datimI
datim.total_datimII
datim.total_datimIII
datim.total_datimIV
datim.total_datim_general
AgywPrev.datim_titleI()
datim.datim_agyw_prevI()
AgywPrev.datim_titleII()
datim.datim_agyw_prevII()
AgywPrev.datim_titleIII()
datim.datim_agyw_prevIII()
AgywPrev.datim_titleIV()
datim.datim_agyw_prevIV()

12757

12526

'DATIM'

5462

5361

...

,Age,0-6 months,07-12 months,13-24 months,25+ months,Total
0,10-14,1378,1180,943,336,3837
1,15-19,77,162,385,55,679
2,20-24,84,212,505,44,845


'Number of individual AGYW that have fully completed at least one DREAMS primary package of services but not the full primary package of services.'

,Age,0-6 months,07-12 months,13-24 months,25+ months,Total
0,10-14,125,193,101,24,443
1,15-19,83,228,138,72,521
2,20-24,34,132,80,37,283


'Number of AGYW that have started a DREAMS service/intervention but have not yet completed it.'

,Age,0-6 months,07-12 months,13-24 months,25+ months,Total
0,10-14,103,90,19,13,225
1,15-19,37,74,12,32,155
2,20-24,27,24,7,18,76


# Communes view

In [ ]:
class AgywPrevCommune(AgywPrev):
    """A class that extend AgywPrev with the purpose of the indicator AGYW_PREV DATIM by commune"""
    __who_am_I ="DATIM"
    def __init__(self,name):
        self.__name = name
        self.__i_am = f"{AgywPrevCommune.__who_am_I} {self.__name}"
        super().__init__(self.__name)
        
    @property
    def who_am_i(self):
        return self.__i_am
    
    def __repr__(self):
        return f"<AgywPrevCommune {self.__i_am}>"
    
    def __str__(self):
        return f"<AgywPrevCommune {self.__i_am}>"
      
        

%%capture
def name_handler(s):
    s = sub(r"[^\w\s]", '', s)
    # Replace all runs of whitespace with a single dash
    s = sub(r"\s+", '_', s)
    return s

datimcommunes = []

for columns in DREAMS_MASTERSHEET.actual_commune.unique():
    globals()[f"datim_{name_handler(columns)}"] = AgywPrevCommune(f"{columns}")
    datimcommunes.append(globals().get(f"datim_{name_handler(columns)}"))



        
        

        

# Verification

In [44]:
datim.data_dreams_valid.to_excel('datim_verification.xlsx',index=False,na_rep="")